<a href="https://colab.research.google.com/github/Dhruvarora0309/NLP_Project/blob/main/AI_in_Industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install moviepy
!pip install gradio

In [ ]:
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration
from moviepy.editor import VideoFileClip
from PIL import Image
import os




# CAPTION GENERATOR

# Step 1: Extract frames from the video
def extract_frames(video_path, output_dir="frames", frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video = VideoFileClip(video_path)
    frames = []
    for i, frame in enumerate(video.iter_frames(fps=frame_rate)):
        frame_path = os.path.join(output_dir, f"frame_{i}.jpg")
        img = Image.fromarray(frame)
        img.save(frame_path)
        frames.append(frame_path)

    return frames

# Step 2: Generate captions for each frame
def captions_helper(frames):
    """
    Generate captions for a list of image frames using the BLIP model.
    :param frames: List of paths to image frames.
    :return: Combined caption string.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    captions = []
    for frame_path in frames:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(image, return_tensors="pt")
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    # Clean up frames directory
    for frame in frames:
        os.remove(frame)

    return " ".join(captions)


# Step 3: Main function
def generate_caption(video_path):
    print("Extracting frames from video...")
    frames = extract_frames(video_path)

    print("Generating captions...")
    captions = captions_helper(frames)

    print("\nGenerated Captions:")
    return captions

# Provide the path to the video file
# video_path = "gymnast.mp4"
# main(video_path)

# def generate_caption(video_path):
    # return ["This is where the captions are supposed to go"]

def generate_video(captions,video_path):
    return video_path




def process_input_video(video_path):

    caption = generate_caption(video_path)
    output_video = generate_video(caption,video_path)

    return video_path,output_video, caption


interface = gr.Interface(
    fn = process_input_video,
    inputs = gr.Video(label="Upload Video"),
    outputs=[
        gr.Video(label="Original Video"),
        gr.Video(label="Generated Video"),
        gr.Textbox(label = "Generated Caption")
        ],
    title="Video Generator",
    description="Upload video, get ai generated counterpart",
)

interface.launch()





Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://542e66d0b04929a775.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Summary Vala

In [ ]:
!pip install gradio

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from moviepy.editor import VideoFileClip
from PIL import Image
import os
import gradio as gr
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video


# Step 1: Extract frames from the video
def extract_frames(video_path, output_dir="frames", frame_rate=1):
    """
    Extract frames from a video file at the specified frame rate (fps).
    :param video_path: Path to the video file.
    :param output_dir: Directory to save extracted frames.
    :param frame_rate: Frames per second to extract.
    :return: List of frame file paths.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video = VideoFileClip(video_path)
    frames = []
    for i, frame in enumerate(video.iter_frames(fps=frame_rate)):
        frame_path = os.path.join(output_dir, f"frame_{i}.jpg")
        img = Image.fromarray(frame)
        img.save(frame_path)
        frames.append(frame_path)

    return frames

# Step 2: Generate captions for each frame
def generate_captions(frames):
    """
    Generate captions for a list of image frames using the BLIP model.
    :param frames: List of paths to image frames.
    :return: Combined caption string.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    captions = []
    for frame_path in frames:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(image, return_tensors="pt")
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    # Clean up frames directory
    for frame in frames:
        os.remove(frame)

    return " ".join(captions)

# Step 3: Summarize the captions
def summarize_captions(captions):
    """
    Summarize the generated captions using a summarization model and format the output as a single, concise sentence.
    :param captions: String containing all generated captions.
    :return: Summarized caption string as a single concise sentence.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(captions, max_length=30, min_length=10, do_sample=False)  # Adjusting length for conciseness
    summarized_text = summary[0]["summary_text"]

    # Post-process to create a single short sentence
    sentences = summarized_text.split(". ")  # Split into individual sentences
    short_summary = " and ".join(sentences[:2])  # Combine only the first two main ideas
    short_summary = short_summary.strip(" .") + "."  # Clean up trailing punctuation or spaces

    return short_summary



# Step 4: Main function
def generate_caption(video_path):
    print("Extracting frames from video...")
    frames = extract_frames(video_path)

    print("Generating captions...")
    captions = summarize_captions(frames)

    print("\nGenerated Captions:")
    # print(captions)

    print("\nSummarizing Captions...")
    summary = summarize_captions(captions)
    print("\nSummary:")

    return summary

# Provide the path to the video file
def generate_video(prompt):
    pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()

    # memory optimization
    pipe.enable_vae_slicing()

    # prompt = "A woman is sitting on the floor in a room and a man is walking with a cart."
    video_frames = pipe(prompt, num_frames=64).frames[0]
    video_path = export_to_video(video_frames)
    print("Video saved to ",video_path)
    return video_path



def process_input_video(video_path):

    caption = generate_caption(video_path)
    output_video = generate_video(caption)

    return video_path,output_video, caption


interface = gr.Interface(
    fn = process_input_video,
    inputs = gr.Video(label="Upload Video"),
    outputs=[
        gr.Video(label="Original Video"),
        gr.Video(label="Generated Video"),
        gr.Textbox(label = "Generated Caption")
        ],
    title="Video Generator",
    description="Upload video, get ai generated counterpart",
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67a71d1d17f53e6c79.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Testing

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from moviepy.editor import VideoFileClip
from PIL import Image
import os

# Step 1: Extract frames from the video
def extract_frames(video_path, output_dir="frames", frame_rate=1):
    """
    Extract frames from a video file at the specified frame rate (fps).
    :param video_path: Path to the video file.
    :param output_dir: Directory to save extracted frames.
    :param frame_rate: Frames per second to extract.
    :return: List of frame file paths.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video = VideoFileClip(video_path)
    frames = []
    for i, frame in enumerate(video.iter_frames(fps=frame_rate)):
        frame_path = os.path.join(output_dir, f"frame_{i}.jpg")
        img = Image.fromarray(frame)
        img.save(frame_path)
        frames.append(frame_path)

    return frames

# Step 2: Generate captions for each frame
def generate_captions(frames):
    """
    Generate captions for a list of image frames using the BLIP model.
    :param frames: List of paths to image frames.
    :return: Combined caption string.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    captions = []
    for frame_path in frames:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(image, return_tensors="pt")
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    # Clean up frames directory
    for frame in frames:
        os.remove(frame)

    return " ".join(captions)

# Step 3: Summarize the captions
def summarize_captions(captions):
    """
    Summarize the generated captions using a summarization model and format the output as a single, concise sentence.
    :param captions: String containing all generated captions.
    :return: Summarized caption string as a single concise sentence.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(captions, max_length=30, min_length=10, do_sample=False)  # Adjusting length for conciseness
    summarized_text = summary[0]["summary_text"]

    # Post-process to create a single short sentence
    sentences = summarized_text.split(". ")  # Split into individual sentences
    short_summary = " and ".join(sentences[:2])  # Combine only the first two main ideas
    short_summary = short_summary.strip(" .") + "."  # Clean up trailing punctuation or spaces

    return short_summary

# Main function
def sumariser(video_path):
    print("Extracting frames from video...")
    frames = extract_frames(video_path)

    print("Generating captions...")
    captions = generate_captions(frames)

    print("\nGenerated Captions:")
    print(captions)

    print("\nSummarizing Captions...")
    summary = summarize_captions(captions)
    print("\nSummary:")
    return summary


# caps = sumariser('/content/00X3U.mp4')
# print(caps)


def generate_new_video(prompt):
    pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()

    # memory optimization
    pipe.enable_vae_slicing()

    # prompt = "A woman is sitting on the floor in a room and a man is walking with a cart."
    video_frames = pipe(prompt, num_frames=64).frames[0]
    new_video_path = export_to_video(video_frames)
    print("Video saved to ",new_video_path)
    return new_video_path

def generate_video(caption):
    file_path = generate_new_video(caption)
    return file_path

def process_input_video(video_path):

    caption = sumariser(video_path)
    output_video = generate_video(caption)

    return video_path,output_video, caption


interface = gr.Interface(
    fn = process_input_video,
    inputs = gr.Video(label="Upload Video"),
    outputs=[
        gr.Video(label="Original Video"),
        gr.Video(label="Generated Video"),
        gr.Textbox(label = "Generated Caption")
        ],
    title="Video Generator",
    description="Upload video, get ai generated counterpart",
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ab5de6e08f078eb2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Little test

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from moviepy.editor import VideoFileClip
from PIL import Image
import os
import gradio as gr
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video
import shutil
import os

def extract_frames(video_path, output_dir="frames", frame_rate=1):
    """
    Extract frames from a video file at the specified frame rate (fps).
    :param video_path: Path to the video file.
    :param output_dir: Directory to save extracted frames.
    :param frame_rate: Frames per second to extract.
    :return: List of frame file paths.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video = VideoFileClip(video_path)
    frames = []
    for i, frame in enumerate(video.iter_frames(fps=frame_rate)):
        frame_path = os.path.join(output_dir, f"frame_{i}.jpg")
        img = Image.fromarray(frame)
        img.save(frame_path)
        frames.append(frame_path)

    return frames

# Step 2: Generate captions for each frame
def generate_captions(frames):
    """
    Generate captions for a list of image frames using the BLIP model.
    :param frames: List of paths to image frames.
    :return: Combined caption string.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    captions = []
    for frame_path in frames:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(image, return_tensors="pt")
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    # Clean up frames directory
    for frame in frames:
        os.remove(frame)

    return " ".join(captions)

# Step 3: Summarize the captions
def summarize_captions(captions):
    """
    Summarize the generated captions using a summarization model and format the output as a single, concise sentence.
    :param captions: String containing all generated captions.
    :return: Summarized caption string as a single concise sentence.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(captions, max_length=30, min_length=10, do_sample=False)  # Adjusting length for conciseness
    summarized_text = summary[0]["summary_text"]

    # Post-process to create a single short sentence
    sentences = summarized_text.split(". ")  # Split into individual sentences
    short_summary = " and ".join(sentences[:2])  # Combine only the first two main ideas
    short_summary = short_summary.strip(" .") + "."  # Clean up trailing punctuation or spaces

    return short_summary

# Main function
def sumariser(video_path):
    print("Extracting frames from video...")
    frames = extract_frames(video_path)

    print("Generating captions...")
    captions = generate_captions(frames)

    print("\nGenerated Captions:")
    print(captions)

    print("\nSummarizing Captions...")
    summary = summarize_captions(captions)
    print("\nSummary:")
    return summary


def generate_video(prompt):
    pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()

    # memory optimization
    pipe.enable_vae_slicing()

    # prompt = "A woman is sitting on the floor in a room and a man is walking with a cart."
    video_frames = pipe(prompt, num_frames=64).frames[0]
    new_video_path = export_to_video(video_frames)
    print("Video saved to ",new_video_path)
    return new_video_path



def move_and_rename_file(src_path, dest_dir, new_name):

    # Ensure the destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Construct the destination path with the new name
    dest_path = os.path.join(dest_dir, new_name)

    # Move and rename the file
    shutil.move(src_path, dest_path)
    print(f"File moved and renamed to: {dest_path}")

    return dest_path

def get_video(captions):
    video_path_new = generate_video(captions)

    src_file = video_path_new
    dest_directory = "/tmp/gradio/5c19e50d905bd3edd2bd9875abac89be9861a3f6a805d4dc8b78ab178eb3503b"
    new_filename = "new_video.mp4"

    moved_file_path = move_and_rename_file(src_file, dest_directory, new_filename)

    return moved_file_path





def process_input_video(video_path):

    caption = sumariser(video_path)
    output_video = get_video(caption)

    return video_path,output_video, caption


interface = gr.Interface(
    fn = process_input_video,
    inputs = gr.Video(label="Upload Video"),
    outputs=[
        gr.Video(label="Original Video"),
        gr.Video(label="Generated Video"),
        gr.Textbox(label = "Generated Caption")
        ],
    title="Video Generator",
    description="Upload video, get ai generated counterpart",
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c08e549fe865556ee7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def generate_video(prompt):
    pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()

    # memory optimization
    pipe.enable_vae_slicing()

    # prompt = "A woman is sitting on the floor in a room and a man is walking with a cart."
    video_frames = pipe(prompt, num_frames=64).frames[0]
    new_video_path = export_to_video(video_frames, output_path='/content/new_video.mp4')
    print("Video saved to ",new_video_path)
    return new_video_path



generate_video("Spider man surfing")

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: export_to_video() got an unexpected keyword argument 'output_path'

In [ ]:
import shutil
import os

def move_and_rename_file(src_path, dest_dir, new_name):
    """
    Move a file from one directory to another and rename it.
    :param src_path: Path to the source file.
    :param dest_dir: Destination directory.
    :param new_name: New name for the file at the destination.
    :return: Full path to the moved and renamed file.
    """
    # Ensure the destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Construct the destination path with the new name
    dest_path = os.path.join(dest_dir, new_name)

    # Move and rename the file
    shutil.move(src_path, dest_path)
    print(f"File moved and renamed to: {dest_path}")

    return dest_path

# Example usage
src_file = "/tmp/tmpl99dzmqv.mp4"
dest_directory = "/content"
new_filename = "new_video.mp4"

moved_file_path = move_and_rename_file(src_file, dest_directory, new_filename)
print(f"Moved file path: {moved_file_path}")


File moved and renamed to: /content/new_video.mp4
Moved file path: /content/new_video.mp4


# Running Code

In [5]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from moviepy.editor import VideoFileClip
from PIL import Image
import os
import gradio as gr
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video
import shutil
import os

def extract_frames(video_path, output_dir="frames", frame_rate=1):
    """
    Extract frames from a video file at the specified frame rate (fps).
    :param video_path: Path to the video file.
    :param output_dir: Directory to save extracted frames.
    :param frame_rate: Frames per second to extract.
    :return: List of frame file paths.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video = VideoFileClip(video_path)
    frames = []
    for i, frame in enumerate(video.iter_frames(fps=frame_rate)):
        frame_path = os.path.join(output_dir, f"frame_{i}.jpg")
        img = Image.fromarray(frame)
        img.save(frame_path)
        frames.append(frame_path)

    return frames

# Step 2: Generate captions for each frame
def generate_captions(frames):
    """
    Generate captions for a list of image frames using the BLIP model.
    :param frames: List of paths to image frames.
    :return: Combined caption string.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    captions = []
    for frame_path in frames:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(image, return_tensors="pt")
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    # Clean up frames directory
    for frame in frames:
        os.remove(frame)

    return " ".join(captions)

# Step 3: Summarize the captions
def summarize_captions(captions):
    """
    Summarize the generated captions using a summarization model and format the output as a single, concise sentence.
    :param captions: String containing all generated captions.
    :return: Summarized caption string as a single concise sentence.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(captions, max_length=30, min_length=10, do_sample=False)  # Adjusting length for conciseness
    summarized_text = summary[0]["summary_text"]

    # Post-process to create a single short sentence
    sentences = summarized_text.split(". ")  # Split into individual sentences
    short_summary = " and ".join(sentences[:2])  # Combine only the first two main ideas
    short_summary = short_summary.strip(" .") + "."  # Clean up trailing punctuation or spaces

    return short_summary

# Main function
def sumariser(video_path):
    print("Extracting frames from video...")
    frames = extract_frames(video_path)

    print("Generating captions...")
    captions = generate_captions(frames)

    print("\nGenerated Captions:")
    print(captions)

    print("\nSummarizing Captions...")
    summary = summarize_captions(captions)
    print("\nSummary:")
    return summary


def generate_video(prompt):
    pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()

    # memory optimization
    pipe.enable_vae_slicing()

    # prompt = "A woman is sitting on the floor in a room and a man is walking with a cart."
    video_frames = pipe(prompt, num_frames=64).frames[0]
    new_video_path = export_to_video(video_frames)
    print("Video saved to ",new_video_path)
    return new_video_path



def move_and_rename_file(src_path, dest_dir, new_name):
  # Ensure the destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Construct the destination path with the new name
    dest_path = os.path.join(dest_dir, new_name)

    # Move and rename the file
    shutil.move(src_path, dest_path)
    print(f"File moved and renamed to: {dest_path}")

    return dest_path

def get_video(captions):
    video_path_new = generate_video(captions)

    src_file = video_path_new
    dest_directory = "/tmp/gradio/5c19e50d905bd3edd2bd9875abac89be9861a3f6a805d4dc8b78ab178eb3503b"
    new_filename = "new_video.mp4"

    moved_file_path = move_and_rename_file(src_file, dest_directory, new_filename)

    return moved_file_path





def process_input_video(video_path):

    caption = sumariser(video_path)
    output_video = get_video(caption)

    return video_path,output_video, caption


interface = gr.Interface(
    fn = process_input_video,
    inputs = gr.Video(label="Upload Video"),
    outputs=[
        gr.Video(label="Original Video"),
        gr.Video(label="Generated Video"),
        gr.Textbox(label = "Generated Caption")
        ],
    title="Video Generator",
    description="Upload video, get ai generated counterpart",
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ad338d8d3d2981661.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
